In [1]:
import pickle
from PIL import Image
import numpy as np
train_path = r"C:\Users\user\Desktop\UVA\AML\KAGGLE"
import PIL
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import matplotlib.pyplot as plt
from torchvision import transforms, utils

import warnings
warnings.filterwarnings("ignore")


In [2]:
train_img = pickle.load(open(train_path+r"\train_img.pkl","rb"))

In [3]:
labels=pickle.load(open(r"C:\Users\user\Desktop\UVA\AML\KAGGLE\train_lbs.pkl","rb"))


In [4]:
print(set(labels))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120}


First, we create two functions to help us go easily from pickle to array and backwards

In [5]:
def pickleToArray(images):
    proc_arrays=[]
    for i in range(len(images)):
        proc_arrays.append(np.array(images[i]))
    proc_arrays=np.array(proc_arrays) 
    return proc_arrays

In [6]:
def arrayToPickle(images):
    proc_pickle=[]
    for i in range(len(images)):
        proc_pickle.append(PIL.Image.fromarray(images[i]))
    return proc_pickle

# Rescale and Random Cropping

In [7]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size

    def __call__(self, image):
        h, w = np.array(image).shape
        new_h, new_w = self.output_size
        img = image.resize((new_h, new_w),Image.ANTIALIAS)
        return img

#Above is basically what you ve already done but with a more beautiful way :-P 
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size

    def __call__(self, image):
        image=np.array(image)
        h, w = image.shape
        new_h, new_w = self.output_size
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]
        return PIL.Image.fromarray(image)
#Not sure if random cropping is good, maybe it's better if it is specific around there where there's significant difference

# Overall Procedure
## 1.Rescaling
We simply resize to (64,64).
## 2.Standardizing
We will standardize per pixel, which means substracting the mean and dividing by standard deviation of each pixel position.
## 3.Flipping
We will flip around the image to augment the data.

In [8]:
processed_train_img = []
pr_labels=[]
size=64
i=0
for image in train_img:
    processed_train_img.append(Rescale((size,size))(image))
    pr_labels.append(labels[i])
    
    i=i+1

In [9]:
len(processed_train_img)

24204

In [10]:
proc_arrays=pickleToArray(processed_train_img)
for i in range(64):
    for j in range(64):
        mean=np.mean(proc_arrays[:,i,j])
        sdev=np.std(proc_arrays[:,i,j])
        proc_arrays[:,i,j]=(proc_arrays[:,i,j]-mean)/sdev


In [11]:
processed_train_img=arrayToPickle(proc_arrays)

In [12]:
len(processed_train_img)

24204

In [13]:
def flip1(image):
    return PIL.Image.fromarray(np.flip(np.array(image),0))
def flip2(image):
    return PIL.Image.fromarray(np.flip(np.array(image),1))
def flip3(image):
    return PIL.Image.fromarray(np.flip(np.flip(np.array(image),0),1))


In [14]:
n=len(processed_train_img)
for i in range(n):
    image=processed_train_img[i]
    processed_train_img.append(flip1(image))
    processed_train_img.append(flip2(image))
    processed_train_img.append(flip3(image))
    pr_labels.append(pr_labels[i])
    pr_labels.append(pr_labels[i])
    pr_labels.append(pr_labels[i])
    
    

In [15]:
len(processed_train_img)

96816

In [ ]:
pickle.dump( processed_train_img, open(r"C:\Users\user\Desktop\UVA\AML\KAGGLE\processed_train_images_2\processed_train_images_2.pkl", "wb" ) )


In [ ]:
pickle.dump(pr_labels,open(r"C:\Users\user\Desktop\UVA\AML\KAGGLE\pr_labels\pr_labels.pkl","wb"))